# Multi-agent Collaboration - News Writer
When you need more than a single agent to handle a complex task, you can create additional specialized agents to address different aspects of the process. However, managing these agents becomes technically challenging as tasks grow in complexity. As a developer using open source solutions, you may find yourself navigating the complexities of agent orchestration, session handling, memory management, and other technical aspects that require manual implementation.

With the multi-agent collaboration capability on Strands Agents SDK, specialized agents work within their domain of expertise, coordinated by a supervisor agent. The supervisor breaks down requests, delegates tasks, and consolidates outputs into a final response. For example, an investment advisory multi-agent system might include agents specialized in financial data analysis, research, forecasting, and investment recommendations. Similarly, a retail operations multi-agent system could handle demand forecasting, inventory allocation, supply chain coordination, and pricing optimization.

In this lab, we will build a multi-agent system where facts about a news event collected by a journalist are used to generate a news story. As shown in the diagram below, multiple agents will be responsible for tasks which are orchestrated by the supervisor agent:

<img src="../../imgs/multi_agent_colloboration_strands.png" width="600">

The workflow shown in the diagram above is as follows:

1. A journalist submits facts to a front-end backed by an LLM (Interface Supervisor)
2. The Interface Supervisor agent sends the facts to a Research agent.
3. The Research agent is equipped with a Tool that does the following:
   1. Entity Extraction: These can be people, companies, products, etc.
   2. Gather background information: This uses the Bedrock Knowledge Base we created in the setup phase. If any entity has low confidence scores, i.e. not mentioned anywhere in the Knowledge Base it is discarded.
5. The Lambda then returns the research to the Research agent, which returns it to the Interface Supervisor agent.
6. Once additional context has been provided by the Research agent, the Interface Supervisor agent sends the research and the facts to the Article Generation agent. This agent is part of a reflection pattern we covered earlier (Lab 5):
   1. News Generation agent: This writes the main news article based on the information provided by the Research agent.
   2. Article Reviewer agent: This provides feedback to the News Generation agent and together, these agents iteratively improve the quality of the generated article.
7. The remainder of the architecture is shown for completeness, and won't be part of this lab. Feel free to implement that if you have time at the end.

Please note that this is a simplified architecture to demonstrate multi-agent collaboration, a complete architecture would incorporate storing outputs at every stage for monitoring agents, and more opportunity for human-in-the-loop capability.

# Let's get started!

In [ ]:
%store -r

<div class="alert alert-info"> 💡 The following cell imports Strands agent packages into the kernel. Sometimes, the library might not be updated from the installation. If you run into any issues with importing the following libraries, simply restart the kernel again and rerun the notebook. 
</div>

In [ ]:
# Install Strands Agents if not already installed
%pip install strands-agents strands-agents-tools -Uq

## Importing Strands Agents
In the following section, we'll import the necessary modules from Strands Agents to create our article generation agent.

In [ ]:
from strands import Agent, tool
from strands_tools import retrieve
from strands.models import BedrockModel

In [ ]:
import logging

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()])

## AWS Service Setup

In [ ]:
import boto3
import time

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region)


# Research Agent Implementation

The following code block creates a specialized agent to gather comprehensive research on entities mentioned in news facts. This research agent extracts 
key entities (people, organizations, products) and retrieves relevant background information from knowledge bases to provide context for article 
generation.

Strands Agent SDK provides a @tool decorator to make it easy for developers to create tools. Detailed descriptions about the tool are embedded in the 
function description. The description is used by Strands Agent to determine the right tool to invoke within the agent loop. For more information about how
to create tools, please refer to this [link](https://strandsagents.com/latest/user-guide/concepts/tools/tools_overview/)

## Entity Extraction Tool for News Research

We're building an entity extraction tool that identifies key information from news facts. This specialized component uses the Strands Agent 
SDK to transform unstructured news content into organized data.

In [ ]:
def extract_entities(news_facts: str) -> str:
    """
    Extract entities (people, organizations, products) from news facts.
    
    Args:
        news_facts: The news facts to extract entities from
        
    Returns:
        Extracted entities in markdown format
    """

    # Create a BedrockModel with specific configuration
    entity_extraction_model = BedrockModel(
        model_id="us.amazon.nova-lite-v1:0",
        region_name=region,
        temperature=1.0,
        max_tokens=2048
    )
    
    extract_agent = Agent(
        model=entity_extraction_model,
        system_prompt="""Your primary function is to extract entities (specifically people, companies/organizations, and products) from news facts collected by a journalist at a news event and output them in consistent markdown format. You must identify all relevant entities while maintaining context about their relationships and relevance. Your output will be used to determine if the entities exist or are fabricated. It is important that your output is in markdown format only.

Input Processing:
1. Accept text input of any length from journalists
2. Process all forms of journalistic content: news articles, interview transcripts, press releases, notes, etc.
3. Maintain the original text for reference and context

Entity Recognition Methodology:
1. Extract full names of people (first, middle, last) where available
2. Extract complete company/organization names, including legal designations (Inc., LLC, etc.)
3. Recognize and extract aliases, nicknames, and abbreviated forms of entities
4. Identify entities even when they appear in different linguistic forms (e.g., possessive, plural)
5. Consider contextual information to accurately identify entities (titles, roles, locations)

Entity Classification:
Categorize each entity as either:
PERSON: Individual human beings
ORGANIZATION: Companies, corporations, agencies, institutions, etc.
PRODUCT: products, items, goods, etc.

Assign specific sub-classifications when possible:
For PERSON: Political figure, executive, celebrity, expert, etc.
For ORGANIZATION: Corporation, government agency, non-profit, educational institution, etc.
For PRODUCT: type, industry

Contextual Information Extraction:
For each entity, extract and preserve:
1. Contextual role/title (CEO, President, Senator, etc.)
2. Associated organizations (for people)

Confidence Scoring:
Assign a confidence score (0.0-1.0).

Limitations and Boundaries
1. Do not attempt to verify entities' existence (this is for the downstream agent)
2. Do not add information about entities not present in the input text
3. Do not make assumptions about entities beyond what is explicitly or implicitly stated
4. Focus only on people and organizations; ignore other entity types

Output Format
Structure output in two sections in consistent markdown format. The first section will be titled "Entities" and it will be a list of the entities that you have identified. Each entity should only have the following attributes:
entity_id: unique identifier
text: extracted text of the entity
type: PERSON or ORGANIZATION or PRODUCT
subtype: specific classification
confidence: confidence score

The second section will be titled "New Facts" and will be followed by the original news facts.

Do not add preambles to your answer. Make sure your answer is in markdown format."""
    )
    
    # Extract entities using the agent
    result = extract_agent(news_facts)
    return result.message

## Research Query Generator for Entity Information

We're building an research query generator that transforms extracted entities into optimized knowledge base queries. When the system has identified people, organizations, and products from news facts, this tool steps in to craft effective search queries that will 
retrieve the most relevant background information. It analyzes the entity list, prioritizes what's important, and formulates queries designed to maximize the quality and relevance of information returned from our knowledge base

In [ ]:
def create_research_query(entities: str) -> str:
    """
    Create a research query for the entities extracted from news facts.
    
    Args:
        entities: The extracted entities in markdown format
        
    Returns:
        A research query for the knowledge base
    """

    # Create a BedrockModel with specific configuration
    research_query_generation_model = BedrockModel(
        model_id="us.amazon.nova-micro-v1:0",
        region_name=region,
        temperature=0.0,
        top_p=1.0,
        max_tokens=4096
    )
    
    research_query_agent = Agent(
        model=research_query_generation_model,
        system_prompt="""You have been provided a list of entities and news facts about a news event. Create a query for an LLM that asks to find research material on the entities indentified in the news facts.

Skip the preamble, your output should only include the query and nothing else.

Your response must start with:
"Conduct indepth research about only the list of entities under the 'Entities' heading provided in markdown format at the end of this input. Find as much relevant research material of a commercial, personal, financial nature as possible. Focus on all types of information which can help in writing a news article about the entities. Only add information if it is in the knowledge base. If an entity isn't found in your knowledge base, discard it from the output."

To the start, add the following regarding output format:
"Output Format
Structure output in two headings in consistent markdown format. The first heading will be titled "Researched Entities" and it will be a list of only those entities for which you found research information. Each entity should only have the following attributes:
entity_id: unique identifier
text: extracted text of the entity
type: PERSON or ORGANIZATION or PRODUCT
subtype: specific classification
confidence: confidence score
research: 
 - research item 1
 - research item 2
 - more research items

The second heading will be titled "New Facts" and will be followed by the original "News Facts" that were given to you.

Make sure to include proper line breaks by:
1. Using a blank line between paragraphs
2. Adding two spaces at the end of lines where you want a soft line break
3. Using proper markdown syntax for lists, headings, and other elements that require specific line formatting."

Your output must be limited to the query that you've constructed for the LLM and nothing more. Do not add any preamble."""
    )
    
    # Create research query using the agent
    result = research_query_agent(entities)
    return result.message

## Research Agent

Our Research Agent implementation leverages Strands' tool composition pattern to create a specialized agent for news research. The agent uses three key 
tools working in sequence:

1. extract_entities: Parses news facts to identify people, organizations, and products with their associated metadata
2. create_research_query: Transforms extracted entities into optimized knowledge base queries
3. retrieve: Executes these queries against our knowledge base to gather relevant background information

The implementation demonstrates how to use Strands' @tool decorator to create modular components that can be orchestrated by a supervisor agent. We'll 
show how to properly handle context passing between tools, manage tool dependencies, and structure the final research output for downstream consumption by
Article Generation agent.


In [ ]:
@tool
def research_agent(news_facts: str) -> str:
    """
    Research agent that extracts entities from news facts and gathers background information.
    
    Use this tool when you need to gather comprehensive research about people, organizations, 
    and products mentioned in news facts. This tool will extract entities, create optimized
    research queries, and retrieve information from knowledge bases to provide context for
    article generation.
    
    Args:
        news_facts: The news facts to research (raw text from journalist notes)
        
    Returns:
        Structured research information about entities mentioned in the news facts
    """
    print("Research agent processing...")
    
    # Get region and account info for KB access
    session = boto3.session.Session()
    region = session.region_name
    account_id = sts_client.get_caller_identity()["Account"]
    print("STEP 1: EXTRACT ENTITIES")
    extract_entities_response = extract_entities(news_facts)
    entities = extract_entities_response['content'][0]['text']
    # print(entities)
    print("\n\nSTEP 2: CREATE RESEARCH QUERIES")
    create_research_query_response = create_research_query(entities)
    research_query = create_research_query_response['content'][0]['text']
    # print(research_query)
    research_results_response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={ 'text': research_query },
        retrieveAndGenerateConfiguration={
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': lab7_kb_id,
                'modelArn': f"arn:aws:bedrock:us-east-1:{account_id}:inference-profile/us.amazon.nova-micro-v1:0",
                'retrievalConfiguration': {
                    'vectorSearchConfiguration': {
                        'numberOfResults': 5,
                        'overrideSearchType': 'HYBRID'
                    }
                }
            },
            'type': 'KNOWLEDGE_BASE'
        }
    )
    print("\n\nSTEP 3: RETRIEVE AND GENERATE")
    research_results = research_results_response['output']['text']
    print(research_results)
    response = f"<research_results>{research_results}</research_results>"
    return research_results

## Testing the Agent
With all the components in place, let's test out our Research agent. We'll be using the following mock news facts that have been gathered at a news event.

In [ ]:
%%time

news_facts = """NeuraHealth Solutions announced its new medical diagnostic platform called "MediScan" at their annual developer conference yesterday.
The system demonstrated 94% accuracy in early disease detection across a trial of 12,000 patients.
Dr. Eliza Chen, Chief Medical Officer at NeuraHealth, revealed the system was trained on 50 million anonymized patient records.
NeuraHealth CEO Marcus Williams stated the company invested $450 million in research and development over three years.
The platform will be piloted at five major hospital networks starting next month.
Senior Vice President of Product Development, Raj Patel, confirmed that FDA approval is expected by the third quarter.
Initial focus areas include cardiovascular disease, diabetes, and early cancer detection."""

research_data = research_agent(news_facts)

Prints the output from the research agent

In [ ]:
print(research_data)

## Article Generation Agent

The following code block creates a specialized agent to generate an Article based on provided input, and potential feedback from the Reviewer agent
Strands Agent SDK provides a @tool decorator to make it easy for developers to create tools. 
Detailed descriptions about the tool is embedded in the function description. The description is used by Strands Agent to determine the right tool to invoke within the agent loop. For more information about how to create tools, please refer to this [link](https://strandsagents.com/latest/user-guide/concepts/tools/tools_overview/)

In [ ]:
@tool
def article_generation_agent(query: str) -> str:
    """
    Generates a professional news article based on research data and facts.
    
    Use this tool when you need to write a news article based on research data, or when revising an article
    to incorporate feedback from a reviewer.
    
    Args:
        query: The research data, facts, and any feedback for article generation.
        
    Returns:
        A professionally written news article.
    """
    print("Article Generation agent processing...")

    # Create a BedrockModel with specific configuration
    article_generation_model = BedrockModel(
        model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
        region_name=region,
        temperature=1.0,
        top_p=1.0,
        max_tokens=4096
    )
    
    article_writer = Agent(
        model=article_generation_model,
        system_prompt="""You are an expert article writer creating engaging news, sports, and entertainment content tailored to audiences and publication requirements.

## Agent Role
You are an expert journalist who transforms raw news facts and related research material into compelling, professionally written news articles. Your articles must be accurate, engaging, and adhere to high journalistic standards.

## Input Processing
1. Carefully analyze all news facts provided about the event
2. Review all contextual research about entities mentioned in the news facts
3. Identify key information, connections, and the most newsworthy elements
4. Organize information in order of importance (inverted pyramid style)

## Article Creation Guidelines
1. Create a concise, attention-grabbing headline that accurately represents the story
2. Write approximately 800 words (adjust if specifically requested otherwise)
3. Begin with a strong lead paragraph that answers the key questions (who, what, when, where, why, how)
4. Structure the article with the most important information first, followed by supporting details
5. Incorporate relevant context from the research materials where appropriate
6. Use direct quotes from sources when available
7. Maintain a neutral, objective tone throughout
8. Ensure factual accuracy - only use information provided in the input materials
9. Avoid speculation, personal opinions, or unsupported claims
10. Use concise, clear language accessible to general readers
11. Break up text with appropriate paragraphs for readability
12. Include a conclusion that ties the story together or points to future developments
13. NEVER make up any quotes that is not explicitly said by any individual.

## Feedback Integration
1. When receiving feedback from review agents, analyze it thoroughly
2. Make all requested changes that align with journalistic standards
3. Revise for clarity, accuracy, balance, or completeness as directed
4. If feedback contains contradictory requests, prioritize factual accuracy and journalistic ethics
5. Return the revised article with all improvements implemented

## Output Format
The output should consist of only:
1. A headline
2. The article body

Do not include:
- Tags like "Headline:" or "Article:"
- Explanations about your writing process
- Notes about sources or research
- Additional formatting markers
- Thoughts or reflections on the article
- Metadata or structural elements

## Example Output Structure:
Major Discovery Transforms Scientific Understanding

Scientists at Stanford University have announced a breakthrough discovery that challenges existing theories...

[Article continues for approximately 800 words]""",
        tools=[], 
        callback_handler=None
    )
    response = str(article_writer(query))
    formatted_response = f"<article>{response}</article>"               
    return formatted_response

## Testing the agent
At this point, you have successfully created an Article Generation agent with given instructions.
Next, we'll test the agent independently to verify that the agent follows the directions and works as intended.

In [ ]:
research_data

In [ ]:
%%time
draft_article = article_generation_agent(research_data)
print(draft_article)

## Article Reviewer agent

The core functionality of a reflection agent is to perform evaluation and refinement of the output through a feedback loop. In a previous lab (Lab 5), we implemented a PR Reviewer Agent responsible for reviewing PR articles. In this section, we are applying those reflection concepts to implement a News Article Reviewer agent that critically evaluates the generated output, identifies potential errors, inconsistencies, or areas for improvement. During this phase, the agent acts as its own critic, scrutinizing the quality, accuracy, and appropriateness of its response.

In [ ]:
@tool
def article_reviewer_agent(article_text: str) -> str:
    """
    Reviews and provides detailed feedback on news, sports, and entertainment articles.
    
    Use this tool when you need professional feedback on an article's clarity, accuracy,
    engagement, balance, and overall journalistic quality.
    
    Args:
        article_text: The complete article text to be reviewed
        
    Returns:
        Detailed review feedback with specific suggestions for improvement
    """
    print("Article Reviewer agent analyzing...")
    
    # Create a BedrockModel with specific configuration
    article_reviewer_model = BedrockModel(
        model_id="us.amazon.nova-pro-v1:0",
        region_name=region,
        temperature=1.0,
        top_p=1.0,
        max_tokens=4096
    )

    article_reviewer = Agent(
        model=article_reviewer_model,
        system_prompt="""You are a professional article reviewer for news, sports and entertainment. Provides expert analysis to improve clarity, accuracy, engagement and journalistic quality.

You are an AI assistant specialized in reviewing news, sports, and entertainment articles. Your expertise helps journalists and content creators refine their writing for clarity, engagement, and journalistic quality.

You will be provided an article in your input, when reviewing an article, analyze these key elements:

1. **Clarity and Readability**
   - Identify sentences longer than 40 words or spanning multiple lines
   - Flag sentences requiring multiple readings to understand
   - Point out repetitive word usage that weakens impact
   - Suggest ways to make complex information more digestible
   - Analyze paragraph length and structure for optimal readability
   - Check for smooth transitions between ideas and sections

2. **Accuracy and Substantiation**
   - Check for claims that lack proper sourcing or evidence
   - Identify potential factual inconsistencies or errors
   - Flag misleading statistics or improper contextualization of data
   - Suggest where additional verification or expert input might be needed
   - Evaluate the reliability and diversity of cited sources
   - Check dates, names, titles, and other factual details for accuracy

For each issue identified, provide:
- A clear explanation of why it weakens the article
- A specific suggestion for improvement
- Where helpful, a rewritten example demonstrating your suggestion
- A priority level (critical, important, or minor) for each feedback item

Conclude your review with:
- A summary of the article's major strengths
- The 3-5 most important areas for improvement
- An overall assessment of the article's effectiveness

Your feedback should be constructive and actionable, focusing on strengthening the article's journalistic quality and reader experience rather than simply pointing out flaws.""",
        tools=[],
        callback_handler=None,
    )
    
    response = str(article_reviewer(article_text))
    formatted_response = f"<review_feedback>{response}</review_feedback>"
    return formatted_response

In [ ]:
%%time 
review_feedback = article_reviewer_agent(draft_article)
print(review_feedback)

## Interface Supervisor agent

In [ ]:
def interface_supervisor_agent(news_facts: str) -> str:
    """
    Orchestrates a complete news article generation workflow using specialized agents.
    
    This tool coordinates between research, article generation, and review agents to produce
    high-quality news articles from raw facts. It manages the entire workflow including:
    1. Research on entities mentioned in the news facts
    2. Article generation based on research and facts
    3. Article review and improvement through feedback
    
    Args:
        news_facts: Raw news facts collected by a journalist
        
    Returns:
        A professionally written and reviewed news article
    """
    print("Interface Supervisor agent processing...")

    # Create a BedrockModel with specific configuration
    interface_supervisor_model = BedrockModel(
        model_id="us.amazon.nova-premier-v1:0",
        region_name=region,
        temperature=1.0,
        top_p=1.0,
        max_tokens=4096
    )
    
    # Create the interface supervisor agent
    interface_supervisor = Agent(
        model=interface_supervisor_model,
        system_prompt="""You are a supervisor agent responsible for orchestrating a news article generation workflow.
Your role is to only coordinate between agents.

Your task is to carry out an article writing workflow that involves the following:

1. You will be provided news facts from a news event about the article to write. Your task is to submit the unmodified facts to the researchAgent. 
The research agent will provide you with additional research about the entities it identified in the news facts. The result from reseachAgent can be found in the <research_results> XML tag.

2. Once the research agent is finished, submit the research information to the articleWritingAgent, which will create an article from the research and news facts. 

3. The article must be reviewed before returning to the user. The content of the article can be found in <article> XML tag. Use the articleReviewerAgent to perform the review.

4. The review feedback can be found in <review_feedback> XML tag. You should perform the article writing and review iteratively until the reviewerAgent is satisfied with the result. 

Finally, you must return only the final article to the user. Do not provide any preemtive or additional explanation, just return the final article to the user.

# Guidelines:
- Do not modify, summarize, or filter the researchAgent agent's output before passing it to the articleWritingAgent agent.
- When working with the articleReviewAgent, always provide the the article generated by the articleWritingAgent as context without any modifications or summarization.
- Do not edit, rewrite, or enhance the articleWritingAgent agent's output before returning it to the user.

You should iterate between the writing (articleWritingAgent) and review (articleReviewAgent) process to come up with best article.

- You must not iterate the writing and review iteration processes more than 2 times. If you reached the maximum iteration, return the latest draft as the final article.
- If any agent returns an error or incomplete output, notify the user with the exact error message.
- Write your final draft in <final> XML tag.
""",
        tools=[research_agent, article_generation_agent, article_reviewer_agent]
    )
    # Process the news facts through the complete workflow
    result = interface_supervisor(news_facts)
    return result.message, interface_supervisor

# Testing News Writer

Let's test this multi-agent system by providing the Interface Supervisor agent with mock news facts that have been collected by a journalist.

In [ ]:
news_facts = """NeuraHealth Solutions announced its new medical diagnostic platform called "MediScan" at their annual developer conference yesterday.
The system demonstrated 94% accuracy in early disease detection across a trial of 12,000 patients.
Dr. Eliza Chen, Chief Medical Officer at NeuraHealth, revealed the system was trained on 50 million anonymized patient records.
"""

In [ ]:
%%time
interface_supervisor_agent_response, interface_supervisor_agent = interface_supervisor_agent(news_facts)

In [ ]:
print(interface_supervisor_agent_response['content'][0]['text'])

## Session and State
Strands agents maintain conversation context, handle state management, and support persistent sessions across interactions.

Strands agents maintain state in several forms:

Conversation History: The sequence of messages between the user and the agent
Tool State: Information about tool executions and results
Request State: Contextual information maintained within a single request

The following code block prints the conversation history for the agent interactions processed in the previous step.

In [ ]:
interface_supervisor_agent.messages

# Summary

In this lab, we demonstrated how to implement a powerful **Multi-Agent Collaboration** pattern using Strands SDK. This pattern enables complex workflows through the orchestration of specialized agents working together to achieve superior results.

Our implementation consisted of four key components:

1. **Research Agent**: A specialized agent which can extract entities from news facts and provide detailed research on each entity to help write a balanced news article.

2. **Article Generation Agent**: A specialized content creator that produces high-quality articles following detailed journalistic guidelines across multiple dimensions including clarity, accuracy, engagement, structure, and technical excellence.

3. **Article Reviewer Agent**: A professional reviewer that analyzes content across multiple dimensions and provides detailed, constructive feedback with specific improvement suggestions.

4. **Interface Supervisor Agent**: This orchestrates the whole workflow of passing information from the user to the research and generation agents.

The multi-agent collaboration pattern demonstrated several key advantages:

- **Specialized Expertise**: Each agent focuses on its core competency - content creation or critical review
- **Iterative Improvement**: The feedback loop enables continuous refinement of the content
- **Workflow Automation**: The supervisor agent manages the entire process without requiring user intervention
- **Quality Enhancement**: The final output benefits from multiple perspectives and expertise

